# Ejercicio para desarrollar en Clase
# Selección de Características<a class="tocSkip">
## Reconocimiento de Patrones <a class="tocSkip">
### Ingenieria Electrónica <a class="tocSkip">
### Universidad Popular del Cesar <a class="tocSkip">
### Prof.: Jose Ramón Iglesias Gamarra - [https://github.com/joseramoniglesias/](https://github.com/joseramoniglesias/) <a class="tocSkip">
  **joseiglesias@unicesar.edu.co**



# IMPORTANTE:

En este código por favor:

*   No agregar celdas nuevas
*   No agregar librerías nuevas
*   No usar otro nombre de variables
*   No cambiar el clasificador ni el número de vecinos del KNN
*   No hacer gráficas


<font color='red'>Esta tarea será corregida automáticamente, y si no se cumple alguna de las condiciones anteriores, es posible que la tarea no se corrija correctamente.</font>

Si quieres hacer modificaciones y probar estrategias distintas, te invito a hacerlo en otro código (no en éste). <font color='red'>Este código debe modificarse sólo en los lugares indicados.</font>

<font color='orange'> Evaluación:</font>
 Este ejercicio tiene n=16 ítems a evaluar, denotados como Q00, Q01, ... Qn a lo largo del notebook como `<Evaluación Qi>`. En tu evaluación (que recibirás en los próximos días) se indicará cómo han sido cada uno de estos n items. Tu nota en este ejercicio será el número de ítems correctamente hechos dividido por n multiplicado por 100 (en escala de 0 a 100%).


# Ejercicio 06: Selección de Características para la Detección de espinas de salmón
----
ENTREGA: se debe desrrollar en clases, <font color='red'>el nombre del archivo debe coincidir con tu número de alumno/a)</font>.

----
En este ejercicio se realizará la clasificación entre dos clases de "parches" de radiogarfías de filetes de salmón de 100x100 pixeles (en escala de grises):

* Clase 0: Espinas (contiene parches de filetes de salmón con espinas). [Ver imágenes](https://www.dropbox.com/s/8bbb7yzyrjkl0qs/fishbones_0.png?dl=00)

* Clase 1: No-Espinas (contiene parches de filetes de salmón sin espinas) [Ver imágenes](https://www.dropbox.com/s/tdgp3shvk47213c/fishbones_1.png?dl=0)

La base de datos contiene 680 imágenes de 100x100 pixeles (340 imágenes por clase).

**ADVERTENCIA:** Este ejercicio tiene fines pedagógicos sólamente, con la idea de que la solución a este problema pueda ejecutarse en un par de minutos. Un buen detector de espinas sigue esta idea pero con miles de radiografías por clase.

A continuación, en cada PREGUNTA deberás ingresar alguna(s) líneas de código, o completar donde aparezca '<= COMPLETAR AQUI".




# Pregunta 0: Cambiar nombre de archivo

Este archivo debe llamarse `numero de alumnx.ipynb`, si tu número de alumno/a es '12345678J', este código debe llamarse `12345678J.ipynb`. La correción automática busca cada uno de estos archivos (según la lista de alumno/as inscrito/as), si tu código no es encontrado, no se ejecutará y te quedarás sin evaluación.


In [ ]:
print('Espero que hayas cambiado el nombre del archivo... :)')

Espero que hayas cambiado el nombre del archivo... :)


# Setup inicial

Liberías necesarias para que funcione el algoritmo.

In [1]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

try: 
  first_execution
  first_execution = 0
except:
  first_execution = 1

if first_execution:
  from IPython.display import clear_output
  !pip install passlib
  clear_output()
  print('Librerías instaladas.')

Librerías instaladas.


## Setup de librerías

In [2]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#
if first_execution:
  import numpy as np
  import matplotlib.pyplot as plt
  import os.path
  from   sklearn.metrics import confusion_matrix, accuracy_score
  from   sklearn.neighbors import KNeighborsClassifier
  from   tqdm.auto import tqdm
  from   passlib.context import CryptContext
  from   requests import get
  import pickle
  !wget https://www.dropbox.com/s/e7c3c4w3rnza3h5/crypto_0.bin?dl=0
  crypt = CryptContext(
        schemes=["pbkdf2_sha256"],
        default="pbkdf2_sha256",
        pbkdf2_sha256__default_rounds=30000
  )
  with open("crypto_0.bin", "rb") as data:
    [idc_list] = pickle.load(data) 
  !rm crypto_0.bin
  clear_output()
  print('Librerías cargadas.')



Librerías cargadas.


## Funciones necesarias

In [3]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

def num2fixstr(x,d):
  # example num2fixstr(2,5) returns '00002'
  # example num2fixstr(19,3) returns '019'
  st = '%0*d' % (d,x)
  return st

def ImageLoad(prefix,num_char,num_img,echo='off'):
  st   = prefix + num2fixstr(num_char,2) + '_' + num2fixstr(num_img,4) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = plt.imread(st)
  return img

def MatrixStats(X):
  p = np.zeros((6),'int')
  n = len(X.shape)
  if n>0:
    p[0] = X.shape[0]
    if n==2:
      p[1] = X.shape[1]
  p[2] = X.min()
  p[3] = X.max()
  p[4] = X.mean()
  p[5] = X.std()
  return p
  
def CheckMatrix(st,X,ps):
  p = MatrixStats(X)
  d = p-ps
  t = not(np.sum(d)==0)
  if t:
    print(st + ': Revisar, esta matriz presenta errores...')
  else:
    print(st + ': ok')
  return t

def CheckPrint(Y,st=True): 
  if np.sum(Y)>0:
    if st:
      print('Revisar, hay error en al menos uno de estos cálculos, no continuar con las siguientes celdas hasta no corregir este error.')
    else:
      print('Revisar, hay error en al menos uno de los datos personales (no coinciden con los ingresados en la PREGUNTA 0),')
      print('NO SUBIR TAREA ANTES DE CORREGIR ESTE ERROR.')
      print('(tu ejercicio corre el riesgo de no ser corregido)')
  else:
    if st:
      print('Felicitaciones! Los cálculos han sido realizados correctamente.')
    else:
      print('Felicitaciones! Los datos coinciden con los ingresados en la Pregunta 0.')


def CheckFile(wclass):
  st = 'fish_'+num2fixstr(wclass,2)
  sfile = st+'_0340.png'
  x = os.path.isfile('fishbones/'+sfile)
  if x:
      print('Imágenes '+st+'*.png: ok')
  else:
      print('Carga de imágenes '+st+' presentan errores.')
  return not(x)

chk = [[680,10,159,3876,1000,609],[680,0,0,100,50,50],[640,10,-3,4,0,1],[40,10,-2,2,0,0],[640,0,0,100,50,50],[40,0,0,100,50,50],[2,2,2,18,10,7],[5,0,1,9,5,3],[640,5,-3,4,0,1],[40,5,-1,2,0,0],[2,2,2,18,10,8]]

# Separación entre training y testing
def SplitTrainTest(X,y,n):

  K      = np.max(y)+1              # número de clases
  N      = np.int(X.shape[0]/K)     # numeros de muestras por clase
  Ntrain = n*K                      # número de muestras para el training
  Ntest  = K*N-Ntrain               # número de muestras para el testing
  M      = X.shape[1]               # número de características por muestra
  Xtrain = np.zeros((Ntrain,M))     # subset de training
  ytrain = np.zeros((Ntrain),'int') # ground truth del training         
  Xtest  = np.zeros((Ntest,M))      # subset de testing
  ytest  = np.zeros((Ntest),'int')  # ground truth del testing  

  # contadores
  itrain = 0
  itest  = 0
  t      = 0

  for j in range(K):     # para cada clase
    for i in range(N):   # para cada imagen de la clase
      if i<n: # training
        Xtrain[itrain,:] = X[t,:]
        ytrain[itrain] = y[t]
        itrain = itrain+1
      else:  # testing
        Xtest[itest,:] = X[t,:]
        ytest[itest] = y[t]
        itest = itest+1
      t = t+1
  
  return Xtrain,ytrain,Xtest,ytest

# Clasificación usando KNN con 1 vecino
def ClassifierKNN(Xtrain,ytrain,Xtest,ytest,echo ='on'):
  #Xtrain, a, b = normalize(Xtrain)
  #Xtest        = Xtest * a + b
  knn = KNeighborsClassifier(n_neighbors=1)
  knn.fit(Xtrain, ytrain)
  ypred        = knn.predict(Xtest)
  acc          = accuracy_score(ytest,ypred)
  C = confusion_matrix(ytest,ypred)
  if echo == 'on':
    print('Entrenando con '+str(Xtrain.shape[0])+' muestras y probando con '+str(Xtest.shape[0])+' muestras')
    print('Los datos tienen '+str(Xtrain.shape[1])+' features')
    print('Testing Accuracy = '+str(acc*100)+'%')
    print('Matriz de Confusión:')
    print(C)
  return acc,C

def file_verify(idt):
  k = -1
  idt_file = idt+'.ipynb'
  my_filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
  if idt_file==my_filename:
    print('Bien! nombre de archivo coincide con tu número de alumnx.')
    k = 0
  else:
    print('ADVERTENCIA: El nombre de archivo de este código no coincide con la variable NumeroID (número de alumnx). Esta tarea no podrá ser corregida.')
    print('Nombre de archivo: '+my_filename)
    print('  Nombre esperado: '+idt_file)

  return k

def id_verify(idt,idc_list):
  ok = False
  idt = idt.upper()
  k = -1
  n = len(idc_list)
  for i in range(n):
    oki = crypt.verify(idt,idc_list[i])
    ok = ok or oki
    if oki:
      k = i
  if k==-1:
    print('ADVERTENCIA: Este ID no se encuentra registrado, contacte al profesor del curso.')
  else:
    print('Bien! ID '+idt+' registrado en la lista del curso.')
  return k

def verify(idt):
  kID   = id_verify(NumeroID,idc_list)
  kFile = file_verify(NumeroID)
  if kID == -1 or kFile == -1:
    k = 1
  else:
    k = 0
  return k

def FinalCheck(T):
  n = len(T)
  for i in range(n):
    if T[i]==1:
      print('Error: El código presenta error en la evaluación de Q'+num2fixstr(i,2))
  if sum(T)>0:
    print('Este código presenta errores. Es recomendable revisarlo antes de subirlo a Google Classroom.')
  else:
    print('FELICITACIONES!!! El código no presenta errores. Puedes subirlo a Google Classroom.')

print('Funciones necesarias ejecutadas.')

Funciones necesarias ejecutadas.


# Otras Funciones

In [4]:
def jfisher(features, ypred, p=None):
  m = features.shape[1]

  norm = ypred.ravel() - ypred.min()
  max_class = norm.max() + 1

  if p is None:
    p = np.ones(shape=(max_class, 1)) / max_class

  # Centroid of all samples
  features_mean = features.mean(0)

  # covariance within class 
  cov_w = np.zeros(shape=(m, m))

  # covariance between classes
  cov_b = np.zeros(shape=(m, m))

  for k in range(max_class):
    ii = (norm == k)                                   # indices from class k
    class_features = features[ii,:]                    # samples of class k
    class_mean = class_features.mean(0)                # centroid of class k 
    class_cov = np.cov(class_features, rowvar=False)   # covariance of class k

    cov_w += p[k] * class_cov                          # within-class covariance

    dif = (class_mean - features_mean).reshape((m, 1))
    cov_b += p[k] * dif @ dif.T                        # between-class covariance
  try:
    return np.trace(np.linalg.inv(cov_w) @ cov_b)
  except np.linalg.LinAlgError:
    return - np.inf

def lbp(img,hdiv=1, vdiv=1, mapping='nri_uniform',norm=False,names=False):
  if mapping == 'nri_uniform':
    n_bins = 59
    st     = 'LBP'
  else:
    n_bins = 10
    st     = 'LBPri'
  (nv,nh) = (vdiv,hdiv)
  nn  = int(np.fix(img.shape[0]/nv))
  mm  = int(np.fix(img.shape[1]/nh))
  k = 0
  for r in range(0,img.shape[0] - nn+1, nn):
    for c in range(0,img.shape[1] - mm+1, mm):
      w = img[r:r+nn,c:c+mm]
      lbp = local_binary_pattern(w,8,1,mapping)
      (xrc, _) = np.histogram(lbp.ravel(), bins=n_bins,range=(0, n_bins))
      if k==0:
        X = xrc
        k = 1
      else:
        X = np.concatenate((X,xrc))
  if norm:
    X = X/np.linalg.norm(X)
  if names==True:
    Xn = []
    for i in range(vdiv):
      for j in range(hdiv):
        for k in range(n_bins):
          Xn.append(st+'('+str(i)+','+str(j)+')-'+str(k))
    return X,Xn
  else:
    return X

def minmax(X):
  ma = X.max(0)
  mi = X.min(0)
  md    = ma-mi
  a     = 1/ md
  b     = -mi/md
  return X * a + b, a, b

def mean0(X):
  mf = X.mean(0)
  sf = X.std(0)
  a  = 1/sf;
  b  = -mf/sf;
  return X * a + b, a, b  

# PREGUNTA 0: Datos personales

Ingresa tus datos para poder evaluarte, 
<font color='red'>si tus datos son incorrectos, este ejercicio no podrá ser corregido y por lo tanto no tendrás nota en esta actividad.</font>

In [ ]:
NumeroID         = '???'   # <= COMPLETAR AQUI NUMERO DE ALUMNO/A ENTRE APOSTROFES
PrimerNombre     = '???'   # <= COMPLETAR AQUI TU PRIMER NOMBRE
PrimerApellido   = '???'   # <= COMPLETAR AQUI TU PRIMER APELLIDO
Correo           = '???'   # <= COMPLETAR AQUI TU CORREO ELECTRONICO

stid             = PrimerNombre+' '+PrimerApellido+' (ID:'+NumeroID+', email:'+Correo+'):'
print('Registro: '+stid)


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR EL INGRESO DE DATOS PERSONALES
#

T = np.ones((17),'int')
T[0] = verify(NumeroID) # <Evaluación Q00>


# PREGUNTA 1: Carga de base de datos

La base de datos consiste en 2 clases con 340 imágenes por clase. Se almacenan en la carpeta `fishbones` con el formato `fish_xx_nnnn.png`, donde `xx` es el ID de la clase (00 para espinas, 02 para no-espinas) y `nnnn` es el número de la foto de la clase (0001, 0002, ... 0340). Las radiografías son de 100x100 pixeles en escala de grises.


En esta celda debes cargar la base de datos a este ambiente de trabajo, descargando el archivo fishbones.zip que se encuentra en este link: `https://www.dropbox.com/s/0pjl68wbxxoghsw/fishbones.zip`. Una vez descargado el archivo zip, debes descomprimirlo.

AYUDA: usar comando `!wget <url archivo zip>` para descargar el archivo, y el comando `!unzip <archivo zip>` para descomprimirlo.



In [ ]:
# Carga de base de datos

# COMPLETAR AQUI DOS LINEAS DE CODIGO
# Linea 1: comando para bajar archivo zip desde dropbox
# Linea 2: comando para descomprimir el archivo zip bajado en la linea 1
# Al final de la ejecucion debe haberse creado la carpeta fishbones con 680 imágenes


??? # <- COMPLETAR AQUI LINEA 1
??? # <- COMPLETAR AQUI LINEA 2


!rm fishbones.zip
clear_output()
print('Base de datos fishbones cargada.')

Base de datos fishbones cargada.


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CARGA DE LA BASE DE DATOS
#

T[1] = CheckFile(0) # <Evaluación Q01>
T[2] = CheckFile(1) # <Evaluación Q02>
CheckPrint(T[1:3])

# PREGUNTA 2: Extracción de características

En esta pregunta se debe realizar la extracción de características de la imagen `img`. Las características a extraer son las basadas en LBP y se extraen usando los siguientes comandos de la librería PyBalu:

## Cómo extraer características LBP rotación-invairiante:

Con el comando

`f  = lbp(img, hdiv=a, vdiv=b, mapping='uniform')`

se divide la imagen `img` en `a x b` particiones y en cada una de ellas se extrae el descriptor LBP (vector de 10 elementos). En este caso `f` es un vector de `10*a*b` elementos. En este ejercicio trabajaremos con `a = b = 1`, es decir sólo una partición por imagen.



In [ ]:
print('Ejecución de '+stid)

K    = ???    # <= COMPLETAR AQUI EL NUMERO DE CLASES
N    = ???    # <= COMPLETAR AQUI EL NUMERO DE IMAGENES POR CLASE
Mlbp = ???    # <= COMPLETAR AQUI EL NUMERO DE ELEMENTOS DEL VECTOR LBP (INVARIANTE A LA ROTACION)

Xlbp  = np.zeros((K*N,Mlbp))    # K*N muestras (filas), y Mlbp características (columnas)

y     = np.zeros((K*N),'int')   # ground truth (clasificacion ideal)

t = 0
print('Cargando imagenes y extrayendo características...')
for j in range(K):            # para cada clase
  print('Extrayendo características: batch '+str(j+1)+'/'+str(K))
  for i in tqdm(range(N)):                # para cada imagen de la clase
    # Lectura de la imagen
    img    = 255*ImageLoad('fishbones/fish_',j,i+1,echo='off')

    # Extracción de LBP invariante a la rotación  de 1x1 particiones
    Xlbp[t,:] = ??? # <= COMPLETAR AQUI PARA EXTRAER LBP (VER HELP MAS ARRIBA)

    y[t] = j # la muestra t pertenece a la clase j
    t = t+1



In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA EXTRACCiÓN DE CARACTERÍSTICAS
#

T[3] = CheckMatrix('Xlbp',Xlbp,chk[0])     # <Evaluación Q03>
T[4] = CheckMatrix('y   ',y*100,chk[1])    # <Evaluación Q04>
CheckPrint(T[3:5])

# Pregunta 3: Clasificación con todas las características

En todos los experimentos usaremos 320 muestras por clase para training y el resto (20) para testing.

Como primer experimento clasificaremos usando todas las características extraídas, es decir con 10 características. Para la separación train/test usaremos la función `SplitTrainTest`. Para la clasificación usaremos la función `ClassifierKNN`.

<font color='orange'>Se recomienda estudiar bien las dos líneas de código que hacen la normalización.</font>





In [ ]:
# Definición de datos de entrada
X = ???                         # <= COMPLETAR AQUI LA MATRIZ DE LAS CARACTERISTICAS LBP EXTRAIDAS

# Separación de datos de Training/Testing
ntrain = ???                    # <= COMPLETAR AQUI EL NUMERO DE MUESTRAS POR CLASE PARA EL TRAINING
Xtrain,ytrain,Xtest,ytest = ??? # <= COMPLETAR AQUI LA SEPARACIÓN TRAINING/TESTING

# Normalización (paso 1): cada columna del training es escalada a numeros entre 0 y 1
Xtrain, a, b = normalize(Xtrain)

# Normalización (paso 2): cada columna del testing es escalda usando el escalamiento del training
# Observar que los factores de escalamiento (a,b) dependen de los datos del training y no del testing.
Xtest        = Xtest * a + b

# Clasificación y evaluación
acc,C_all = ???                 # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA CLASIFICACIÓN


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA SEPARACIÓN TRAIN/TEST Y SU CLASIFICACIÓN
#

T[5]  = CheckMatrix('X     ',X,chk[0])          # <Evaluación Q05>
T[6]  = CheckMatrix('Xtrain',Xtrain,chk[2])     # <Evaluación Q06>
T[7]  = CheckMatrix('Xtest ',Xtest,chk[3])      # <Evaluación Q07>
T[8]  = CheckMatrix('ytrain',ytrain*100,chk[4]) # <Evaluación Q08>
T[9]  = CheckMatrix('ytest ',ytest*100,chk[5])  # <Evaluación Q09>
T[10] = CheckMatrix('C_all' ,C_all,chk[6])      # <Evaluación Q10>
CheckPrint(T[5:11])

# PREGUNTA 4: Selección de características usando "Búsqueda Exhaustiva"

En este ejemplo buscaremos las q=5 mejores características de las m=10 extraídas. La búsqueda exhaustiva -en este caso- evalúa todas las combinaciones de 5 características (sin repetición), es decir, evalúa las siguientes m!/(q! x (m-q!) = 10!/(5! x (10-5)!) = 252 combinaciones:

* Combinación 1: (0,1,2,3,<font color='orange'> 4 </font> )

* Combinación 2: (0,1,2,3,<font color='orange'> 5 </font>)

* Combinación 3: (0,1,2,3,<font color='orange'> 6 </font>)

* Combinación 4: (0,1,2,3,<font color='orange'> 7 </font>)

* Combinación 5: (0,1,2,3,<font color='orange'> 8 </font>)

* Combinación 6: (0,1,2,3,<font color='orange'> 9 </font>)

* Combinación 7: (0,1,2,<font color='magenta'> 4 </font>,<font color='cyan'> 5 </font>) -> la combinación (0,1,2,3,4,0) no se evalúa porque se repite el 0, además la combinación (0,1,2,4,3) tampoco se evalúa porque ya fue evaluada en la combinación 1 (en otro orden).

* Combinación 8: (0,1,2,<font color='magenta'> 4 </font>,<font color='cyan'> 6 </font>)

* Combinación 9: (0,1,2,<font color='magenta'> 4 </font>,<font color='cyan'> 7 </font>)

* :

* Combinación 252: (5,6,7,8,9)

La búsqueda exhaustiva escoge la "mejor" de estas 252 combinaciones. La mejor combinación será aquella que tenga la mayor "separabilidad" entre las clases. Para obtener un "score" de la separabilidad de las características podemos usar la función de Fisher que mide qué tan compactas están cada una de las clases, y qué tan separadas están entre ellas. Una buena separabilidad (un buen "score") se obtiene si en el espacio de características, las clases son compactas y la separación entre ellas es grande. Este "score" está implementado en la función `jfisher`. Entre más grande el "score" mayor es la separabilidad. En la búsqueda exhaustiva buscamos aquella combinación que maximiza el score entregado por `jfisher`.

## Ejemplo de cómo usar `jfisher`:

A manera de ejemplo, vamos a suponer que hemos almacenado en `Z` la extracción de 20 características de 1000 muestras, es decir el tamaño de esta matriz `Z` es de 1000x20. Adicionalmente, la clasificación ideal de estas 1000 muestras se encuentra almacenada en la variable `d` de 1000 elementos. En este ejemplo, los elementos de `d` toman los valores 0, 1, 2 ó 3 (porque hay 4 clases).

En este ejemplo, deseamos evaluar el "score" de las características (3,15,17). Para evaluar el desempeño de estas tres características seguimos estos cuatro pasos:

* Definimos `i1=3`, `i2=15`, y `i3=17`.

* Definimos `ii = (i1,i2,i3)`.

* Escogemos de `Z` las columnas indexadas por `ii` usando `Z_sel = Z[:,ii]`. Ahora `Z_sel` tiene 1000 muestras y 3 columnas. 

* Para obtener el "score" de estas características seleccionadas, usamos el comando: `score = jfisher(Z_sel,d)`

En este ejemplo, en la variable `score` hemos almacenado la evaluación de la función de Fisher de la separabilidad para la selección de las columas (3,15,17).

En el siguiente código es necesario implementar la búsqueda exhaustiva. El input de este código es la matriz `Xtrain` de 640x10 elementos, y el vector `ytrain` de 640 elementos, la salida debe ser el vector `best_indices` de 5 números distintos entre 0 y 9 (ya que existen 10 características en `Xtrain` y se escogen 5), que corresponde a la mejor combinación de características. En este código se hacen 252 evaluaciones de la función de Fisher, una para cada combinación, y se selecciona aquella combinación que maximiza la función de Fisher.

La búsqueda exhaustiva se realiza con cinco contadores `(i1,i2,i3,i4,i5)` para cada una de las 5 características. Como nuestros datos tienen 10 características, es necesario observar que 

* `i1` debe contar de 0 a 5

* `i2` debe contar de `i1+1` a 6

* `i3` debe contar de `i2+1` a 7

* `i4` debe contar de `i3+1` a 8

* `i5` debe contar de `i4+1` a 9

En esta implemantación, `i1 < i2 < i3 < i4 < i5`, de esta manera 1) se recorren todos los índices sin repetirse, es decir la combinación (1,1,2,3,4) no debe evaluarse, y 2) no se evalúan combinaciones que ya han sido evaluadas anteriormente, es decir si evaluó (1,2,3,4,5), no se evalúa (3,2,1,4,5) ya que ambas combinaciones tienen el mismo "score".


<font color='red'> IMPORTANTE: La ejecución de este algoritmo se aborta si se intenta evaluar la función de Fisher más de 252 veces.



In [ ]:
#######################################################################
# AYUDA: Hay que leer y entender el texto anterior (de la Pregunta 4) #
#######################################################################


m = X.shape[???]              # <= COMPLETAR AQUI EL NÚMERO DE CARACTERÍSTICAS DE X, NO PONER m = 10 
score_max = 0                 # Inicialización del score máximo
best_indices = (-1,-1,-1)     # Inicialización de las columnas seleccionadas
p = 0                         # Contador de evaluaciones
for i1 in range(???):         # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i1 (PRIMERA COLUMNA SELECCIONADA)
  for i2 in range(???):       # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i2 (SEGUNDA COLUMNA SELECCIONADA)
    for i3 in range(???):     # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i3 (TERCERA COLUMNA SELECCIONADA)
      for i4 in range(???):   # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i4 (CUARTA COLUMNA SELECCIONADA)
        for i5 in range(???): # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i5 (QUINTA COLUMNA SELECCIONADA) 
          if p>252:
            break
          p = p+1
          ii = (i1,i2,i3,i4,i5)
          Xtrain_sel = ???    # <= COMPLETAR AQUI CON LAS 5 COLUMNAS DE Xtrain INDICADAS POR ii
          score = ???         # <= COMPLETAR AQUI CON LA FUNCIÓN DE FISHER PARA OBTENER EL SCORE DE Xtrain_sel
          if score > score_max:
            score_max = score
            best_indices = ii
            print('nuevo máximo encontrado: score = '+"{:.4f}".format(score_max)+' para la combinación '+str(ii))
if p==252:
  print('Se evaluaron '+str(p)+' combinaciones.')
  print('Las 5 características seleccionadas son: '+str(best_indices))
else:
  print('*** ADVERTENCIA ***')
  print('El código se ha interrumpido porque ha intentado evaluar más 252 combinaciones.')
  print('Es necesario optimzar este código antes de continuar.')


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA BÚSQUEDA EXHAUSTIVA
#

T[11] = CheckMatrix('best_indices',np.array(best_indices),chk[7])          # <Evaluación Q11>
CheckPrint(T[11])

# PREGUNTA 5: Clasificación con características seleccionadas

A continuación se definen las matrices `Xtrain_sel` (de 640x5 elementos) y `Xtest_sel` (de 40x5) elementos, tomando las columnas seleccionadas de `Xtrain` y `Xtest` respectivamente usando los índices encontrados en `best_indices`.

Finalmente, entrenamos y probamos el clasificador con estos nuevos datos. No es necesario definir `ytrain_sel` y `ytest_sel` porque son iguales a `ytrain` y `ytest` respectivamente

In [ ]:
Xtrain_sel = ??? # <= COMPLETAR AQUI: SELECCIONAR LAS COLUMNAS DE Xtrain
Xtest_sel  = ??? # <= COMPLETAR AQUI: SELECCIONAR LAS COLUMNAS DE Xttest
acc,C_sel  = ??? # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA CLASIFICACIÓN


<font color='orange'> APRENDIZAJE: Si has hecho los pasos correctamente, observarás que con 5 características se obtienen en este problema un mejor desempeño que con 10 características. En esta solución, con selección de características, hemos subido el accuracy en 5%.


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA EXTRACCiÓN DE CARACTERÍSTICAS
#

T[12] = CheckMatrix('Xtrain_sel',Xtrain_sel,chk[8])     # <Evaluación Q12>
T[13] = CheckMatrix('Xtest_sel ',Xtest_sel,chk[9])      # <Evaluación Q13>
T[14] = CheckMatrix('C_sel     ',C_sel,chk[10])      # <Evaluación Q14>
CheckPrint(T[12:15])

# PREGUNTA 6: Verificar datos personales

<font color='red'> **ADVERTENCIA:** En este ejercicio en necesario que revises bien tus datos personales. Si no son correctos, no tendrás nota en este ejercicio.

In [ ]:
# POR FAVOR VUELVE A DIGITAR ESTOS DATOS PARA VERIFICAR QUE NO HAYA ERRORES DE TIPEO
NumeroID_2       = '???' # <= COMPLETAR AQUI NUMERO DE ALUMNO/A ENTRE APOSTROFES
Correo_2         = '???' # <= COMPLETAR AQUI TU CORRECO ELECTRONICO

In [ ]:
#
# EJECUTAR ESTA CELDA PARA VERIFICAR SI LOS DATOS COINCIDEN CON LOS YA INGRESADOS
#

T[15] = NumeroID != NumeroID_2  # <Evaluación Q15>
T[16] = Correo   != Correo_2    # <Evaluación Q16>
CheckPrint(T[15:17],0)

# PASO FINAL: Verificar todas las respuestas y mostar la tarea



In [ ]:
# CHEQUEO FINAL DE TODOS LAS PREGUNTAS
T = np.ones((17),'int')
print('Verificación de ID y nombre de archivo:')
T[0]  = verify(NumeroID)                                                     # <Evaluación Q00>
print('--------------------------------------------------- ')
T[1]  = CheckFile(0)                                                         # <Evaluación Q01>
T[2]  = CheckFile(1)                                                         # <Evaluación Q02>
T[3]  = CheckMatrix('Xlbp                 ',Xlbp,chk[0])                     # <Evaluación Q03>
T[4]  = CheckMatrix('y                    ',y*100,chk[1])                    # <Evaluación Q04>
T[5]  = CheckMatrix('X                    ',X,chk[0])                        # <Evaluación Q05>
T[6]  = CheckMatrix('Xtrain               ',Xtrain,chk[2])                   # <Evaluación Q06>
T[7]  = CheckMatrix('Xtest                ',Xtest,chk[3])                    # <Evaluación Q07>
T[8]  = CheckMatrix('ytrain               ',ytrain*100,chk[4])               # <Evaluación Q08>
T[9]  = CheckMatrix('ytest                ',ytest*100,chk[5])                # <Evaluación Q09>
T[10] = CheckMatrix('C_all                ' ,C_all,chk[6])                   # <Evaluación Q10>
T[11] = CheckMatrix('best_indices         ',np.array(best_indices),chk[7])   # <Evaluación Q11>
T[12] = CheckMatrix('Xtrain_sel           ',Xtrain_sel,chk[8])               # <Evaluación Q12>
T[13] = CheckMatrix('Xtest_sel            ',Xtest_sel,chk[9])                # <Evaluación Q13>
T[14] = CheckMatrix('C_sel                ',C_sel,chk[10])                   # <Evaluación Q14>
T[15] = NumeroID != NumeroID_2                                               # <Evaluación Q15>
T[16] = Correo   != Correo_2                                                 # <Evaluación Q16>
print('--------------------------------------------------- ')
FinalCheck(T)
print('--------------------------------------------------- ')


<font color='red'> **ADVERTENCIA FINAL:** Debes verificar que todas las celdas de este código hayan sido ejecutadas sin error. Si durante la evaluación automática tu código "se cae" en una celda, las siguientes celdas no serán ejecutadas (ni evaludas). 

<font color='orange'> Este archivo, que debe llamarse [tu NumeroID].ipynb, debe mostrarsele al profesor en la actividad Ejercicio 06.
